## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium

## Read-in

In [3]:
df = pd.read_csv('SoFlaTopRentals_013122 - Copy of SoFla_rentals_Realtor_013122.csv')

In [4]:
df = df.head(10)
df = df.filter(items=['type','price_per_month','address','Listing agent '])

## Google Maps API Geolocater Setup

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
df['geo_address'] = df['address']

In [7]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [8]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Correction Section

In [9]:
df.columns

Index(['type', 'price_per_month', 'address', 'Listing agent ', 'geo_address',
       'geocoded', 'lat', 'lon'],
      dtype='object')

In [16]:
df.at

,type,price_per_month,address,Listing agent,geo_address,geocoded,lat,lon
0,For Rent - House,"$350,000","190 Palm Ave, Miami Beach, FL 33139",Maria Tartaglia with Tartaglia International R...,"190 Palm Ave, Miami Beach, FL 33139","(25.780001, -80.1639402)",25.780001,-80.163940
1,For Rent - House,"$350,000","5718 North Bay Road, Miami Beach, FL 33140",David Hunt Solomon with BHHS EWM Realty,"5718 North Bay Road, Miami Beach, FL 33140","(25.8366008, -80.13206339999999)",25.836601,-80.132063
2,For Rent - House,"$350,000","3905 Gem Twist Ct, Wellington, FL 33414","Laurie Elliot, Illustrated Properties","3905 Gem Twist Ct, Wellington, FL 33414","(26.6251799, -80.28302339999999)",26.625180,-80.283023
3,For Rent - House,"$325,000","27 Star Island Drive, Miami Beach, FL 33139",Alyssa Jansheski with The Inside Network,"27 Star Island Drive, Miami Beach, FL 33139","(25.7791558, -80.1500027)",25.779156,-80.150003
4,For Rent - House,"$300,000","28 West Dilido Drive, Miami Beach, FL 33139",Dina Goldentayer with Douglas Elliman,"28 West Dilido Drive, Miami Beach, FL 33139","(25.7868822, -80.1594246)",25.786882,-80.159425
5,For Rent - House,"$250,000","529 Bontona Avenue, Fort Lauderdale, FL 33301",Julie Jones Douglas Elliman,"529 Bontona Avenue, Fort Lauderdale, FL 33301","(26.1154214, -80.1230994)",26.115421,-80.123099
6,For Rent - House,"$250,000","158 Palm Avenue, Miami Beach, FL 33139",Lourdes Alatriste with Douglas Elliman,"158 Palm Avenue, Miami Beach, FL 33139","(25.7792121, -80.16299430000001)",25.779212,-80.162994
7,For Rent - House,"$250,000","30 Palm Avenue, Miami Beach, FL 33139",Jill Eber with NRT-Florida,"30 Palm Avenue, Miami Beach, FL 33139","(25.7775264, -80.1580989)",25.777526,-80.158099
8,For Rent - House,"$250,000","440 South Mashta Drive, Key Biscayne, FL 33149",Gustavo Gambino with Gambino Real Estate Corp.,"440 South Mashta Drive, Key Biscayne, FL 33149","(25.6819574, -80.1711586)",25.681957,-80.171159
9,For Rent - House,"$225,000","317 Ocean Boulevard, Golden Beach, FL 33160",Diane Lieberman with ONE SOTHEBYS INTL. REALTY,"317 Ocean Boulevard, Golden Beach, FL 33160","(25.9638856, -80.1193847)",25.963886,-80.119385


## HTML Popup Formatter

In [10]:
def popup_html(row):
    Address = row['address']
    price_per_month = row['price_per_month']
    List_Agent = row['Listing agent ']
    Type = row['type']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Price Per Month: </strong>{}'''.format(price_per_month) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(List_Agent) + '''<br>
    <strong>Description: </strong>{}'''.format(Type) + '''<br>
    </html>
    '''
    return html

In [11]:
df.columns

Index(['type', 'price_per_month', 'address', 'Listing agent ', 'geo_address',
       'geocoded', 'lat', 'lon'],
      dtype='object')

## Map Maker

In [12]:
### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10)


### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Top Rental Listings in South Florida")

m.get_root().html.add_child(folium.Element(title_html))

for index, row in df.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

# Display map
m

In [13]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [14]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [15]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFla_top_ten_rentals_02072023
